<a href="https://colab.research.google.com/github/RLMelecio/GeoMapper_script_runner/blob/main/Geomapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installed Packages

In [ ]:
!pip install cartopy
!pip install hvplot
!pip install netCDF4 xarray
!pip install metpy
!pip install earthaccess pydap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.9/409.9 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.3/77.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 84.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing in

In [ ]:
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import firebase_admin
from firebase_admin import credentials, firestore
import json
import xarray as xr
import numpy as np
from metpy.cbook import get_test_data
import datetime
import earthaccess
import os

NetCDF Global Sea Surface Temperature map visualization.

In [ ]:
import json

# CODE PART
# EARTH DATA AUTHENTICATION
os.environ['EARTHDATA_USERNAME'] = 'rolandmelecio'
os.environ['EARTHDATA_PASSWORD'] = 'EARTHDATA_user_123'
auth = earthaccess.login(strategy= 'environment')

# SEARCH DATA AND SORT IT TO THE LATEST DATA GRANULE
firstday_currentMonth = datetime.datetime.today().replace(day=1).strftime('%Y-%m-%d')

latestday = datetime.datetime.today().strftime('%Y-%m-%d')

results = earthaccess.search_data(
    short_name='MUR-JPL-L4-GLOB-v4.1',
    temporal=(firstday_currentMonth, latestday),
    sort_key = 'end_date',
    bounding_box=(-10, 20, 10, 50),
    count= 10
)


sorted_results = sorted(results, key=lambda x: datetime.datetime.strptime(x['umm']['TemporalExtent']['RangeDateTime']['EndingDateTime'], '%Y-%m-%dT%H:%M:%S.%fZ'), reverse=True)

download_dir = '/content/my-download-folder'
files = earthaccess.download([sorted_results[0]], download_dir)

print(f"Downloaded files: {files}")



# ================
# print(json.dumps(results[0], indent=2))

QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

Downloaded files: ['/content/my-download-folder/20240905090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc']


TO UPLOAD DATASET COORDINATES AND SST DATA TO FIRESTORE

In [ ]:
# # Initializing dataset
# dataset = xr.open_dataset('20190210000000-NAVO-L4_GHRSST-SST1m-K10_SST-GLOB-v02.0-fv01.0.nc4')
# ds_var = dataset.metpy.parse_cf('analysed_sst')

# # Firebase Admin SDK credentials (Kas-a rani e execute kay mag prompt nig error otherwise)
# # cred = credentials.Certificate('geomapper-d2b26-firebase-adminsdk-ee6he-afa0190bb3.json')
# # firebase_admin.initialize_app(cred)
# # db = firestore.client()

# lat_min, lat_max = 5, 20
# lon_min, lon_max = 115.0, 130.5

# # Filtering the variable and coordinates
# filtered_sst_data = dataset['analysed_sst'].isel(time=0).sel(lat=slice(lat_max, lat_min), lon=slice(lon_min, lon_max)).values
# filtered_lat = dataset['lat'].sel(lat=slice(lat_max, lat_min)).values
# filtered_lon = dataset['lon'].sel(lon=slice(lon_min, lon_max)).values
# filtered_time = dataset['time'].values[0]

# data_to_upload = []

# for i in range(filtered_lat.shape[0]):
#   for j in range(filtered_lon.shape[0]):
#     entry = {
#         "latitude" : float(filtered_lat[i]),
#         "longitude" : float(filtered_lon[j]),
#         "sst" : float(filtered_sst_data[i, j]),
#         "time" : str(filtered_time)
#     }
#     data_to_upload.append(entry)


# # Uploading this data to Firestore
# # for entry in data_to_upload:
# #   db.collection('sea_surface_data').add(entry)


# # Checking if there are existing entries of the looped data from Firestore
# # for entry in data_to_upload:
# #     existing_entries = db.collection('sea_surface_data').where('latitude', '==', entry['latitude']).where('longitude', '==', entry['longitude']).where('time', '==', entry['time']).get()

# #     if not existing_entries:
# #         db.collection('sea_surface_data').add(entry)

TO PLOT NETCDF DATA SST AND DISPLAY IT AS A MAP.

In [ ]:
# dataset = xr.open_dataset('20190210000000-NAVO-L4_GHRSST-SST1m-K10_SST-GLOB-v02.0-fv01.0.nc4')
# ds_var = dataset.metpy.parse_cf('analysed_sst')

# # lat = dataset.lat.values
# # lon = dataset.lon.values
# # sst = dataset['analysed_sst'].isel(time=0).values - 273.15

# lat_min, lat_max = 6.5, 8.0
# lon_min, lon_max = 125.0, 130.5

# # Filtering the variable and coordinates
# filtered_sst_data = dataset['analysed_sst'].isel(time=0).sel(lat=slice(lat_max, lat_min), lon=slice(lon_min, lon_max)).values
# filtered_lat = dataset['lat'].sel(lat=slice(lat_max, lat_min)).values
# filtered_lon = dataset['lon'].sel(lon=slice(lon_min, lon_max)).values

# # Plotting the geospatial map
# fig, ax = plt.subplots(figsize=(5, 5), subplot_kw={'projection': ccrs.PlateCarree()})
# ax.set_title('Sea Surface Temperature around the Philippines')
# im_sst = ax.imshow(filtered_sst_data_celsius, extent=(lon_min, lon_max, lat_min, lat_max),
#                    cmap='jet', origin='upper', transform=ccrs.PlateCarree())
# ax.add_feature(cfeature.LAND, zorder=1, edgecolor='black')
# ax.add_feature(cfeature.COASTLINE, zorder=1)
# ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())

# cbar_sst = plt.colorbar(im_sst, ax=ax, orientation='vertical', pad=0.05, aspect=50)
# cbar_sst.set_label('Sea Surface Temperature (°C)')

# plt.tight_layout()
# plt.show()

TO IDENTIFY VARIABLES NAMES AND DIMENSIONS

In [ ]:
# for var_names in dataset.data_vars:
#   variable = dataset[var_names]
#   print(f"\nVariable: {var_names}")
#   print(f"\nDimensions: {variable.dims}")
#   print(f"Shape: {variable.shape}")

#   print(f"Sample data: {variable.values[:5]}")

# TO CHECK FOR NAN VALUES IN SST DATA

# print("Check for NaN values in SST Data...")
# nan_count = xr.DataArray(sst).isnull().sum()
# total_count = sst.size
# valid_count = total_count - nan_count

# print(f"Total SST Data points: {total_count}")
# print(f"Valid SST data points: {valid_count}")
# print(f"NaN SST data points: {nan_count}")

OLD CODE

In [ ]:

# # Initializing dataset
# dataset = xr.open_dataset('20190210000000-NAVO-L4_GHRSST-SST1m-K10_SST-GLOB-v02.0-fv01.0.nc4')
# ds_var = dataset.metpy.parse_cf('analysed_sst')

# # Firebase Admin SDK credentials (Kas-a rani e execute kay mag prompt nig error otherwise)
# # cred = credentials.Certificate('geomapper-d2b26-firebase-adminsdk-ee6he-afa0190bb3.json')
# # firebase_admin.initialize_app(cred)
# db = firestore.client()

# # lat_min, lat_max = 4, 21 ORGINAL
# # lon_min, lon_max = 116, 127 ORIGINAL

# # Davao Region Area only
# lat_min, lat_max = 6, 8
# lon_min, lon_max = 124, 129

# # Filtering the variable and coordinates
# filtered_sst_data = dataset['analysed_sst'].isel(time=0).sel(lat=slice(lat_max, lat_min), lon=slice(lon_min, lon_max)).values
# filtered_sst_data_celsius = filtered_sst_data - 273.15
# filtered_lat = dataset['lat'].sel(lat=slice(lat_max, lat_min)).values
# filtered_lon = dataset['lon'].sel(lon=slice(lon_min, lon_max)).values
# filtered_time = dataset['time'].values[0]

# data_to_upload = []

# # DEL
# # fig, ax = plt.subplots(figsize=(5, 5), subplot_kw={'projection': ccrs.PlateCarree()})
# # ax.set_title('Sea Surface Temperature on Davao City')
# # im_sst = ax.imshow(filtered_sst_data_celsius, extent=(lon_min, lon_max, lat_min, lat_max),
# #                    cmap='jet', origin='upper', transform=ccrs.PlateCarree())
# # ax.add_feature(cfeature.LAND, zorder=1, edgecolor='black')
# # ax.add_feature(cfeature.COASTLINE, zorder=1)
# # ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())

# # cbar_sst = plt.colorbar(im_sst, ax=ax, orientation='vertical', pad=0.05, aspect=50)
# # cbar_sst.set_label('Sea Surface Temperature (°C)')

# # plt.tight_layout()
# # plt.show()
# # DEL


# for i in range(filtered_lat.shape[0]):
#   for j in range(filtered_lon.shape[0]):
#     sst = float(filtered_sst_data_celsius[i, j])

#     if np.isnan(sst):
#       weight = 0
#     else:
#       if sst >= 28.5:
#         weight = 1
#       elif sst >= 27.5 and sst < 28.5:
#         weight = 0.8
#       elif sst >= 26.5 and sst < 27.5:
#         weight = 0.5
#       elif sst >= 25.5 and sst < 26.5:
#         weight = 0.3
#       else:
#         weight = 0.01

#     entry = {
#         "latitude" : float(filtered_lat[i]),
#         "longitude" : float(filtered_lon[j]),
#         "sst" : sst,
#         "weight": weight,
#         "time" : str(filtered_time)
#     }
#     data_to_upload.append(entry)



# # Uploading this data to Firestore
# for entry in data_to_upload:
#   db.collection('sea_surface_data').add(entry)
# # print(data_to_upload)

# # Checking if there are existing entries of the looped data from Firestore
# # for entry in data_to_upload:
# #     existing_entries = db.collection('sea_surface_data').where('latitude', '==', entry['latitude']).where('longitude', '==', entry['longitude']).where('time', '==', entry['time']).get()

# #     if not existing_entries:
# #         db.collection('sea_surface_data').add(entry)

In [ ]:
# import netCDF4 as nc
# import numpy as np
# import xarray as xr
# import hvplot.xarray
# import matplotlib.pyplot as plt
# import cartopy.crs as crs
# import cartopy.feature as cfeature

# dataset = nc.Dataset('20190210000000-NAVO-L4_GHRSST-SST1m-K10_SST-GLOB-v02.0-fv01.0.nc4', mode='r')

# latitude = dataset.variables['lat'][:]
# longitude = dataset.variables['lon'][:]
# seasurface_temp = dataset.variables['analysed_sst'][0, :, :]

# sst_celcius = seasurface_temp - 273.15

# plt.figure(figsize=(8,8))
# ax = plt.axes(projection=crs.PlateCarree())

# ax.set_extent([120, 130, 5, 10], crs=crs.PlateCarree())
# ax.coastlines(resolution='110m')
# ax.add_feature(cfeature.BORDERS, linestyle=':')
# ax.add_feature(cfeature.LAND, color='green')
# ax.add_feature(cfeature.OCEAN, color='lightblue')


# lon, lat = np.meshgrid(longitude, latitude)

# sst_plot = ax.pcolormesh(lon, lat, sst_celcius, cmap='coolwarm', transform=crs.PlateCarree())

# cbar = plt.colorbar(sst_plot, orientation='horizontal', pad=0.07)
# cbar.set_label('Sea Surface Temperature (°C)')
# plt.title('Global Sea Surface Temperature')
# plt.show()

